dis.008


Import libraries

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob
from matplotlib import pyplot
%matplotlib inline

Define local file locations

In [2]:
local_folder = "C:/Users/Max81007/Desktop/Python/Resource_Watch/Raster/dis.008/"
file_name = "dis_008_vulnerability_to_volcanic_eruptions.asc"

local_orig = local_folder + file_name

orig_extension_length = 4 #4 for each char in .tif
local_edit = local_orig[:-orig_extension_length] + "_edit.tif" 

s3 tools

In [3]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/dis_008_vulnerability_to_volcanic_eruptions/"

s3_file = "dis_008_vulnerability_to_volcanic_eruptions.tif"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"

os.environ["Zs3_key1"] = "s3://wri-public-data/" + s3_key_orig
os.environ["Zs3_key2"] = "s3://wri-public-data/" + s3_key_edit

class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [4]:
files = [local_orig, local_edit]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'AAIGrid', 'dtype': 'int32', 'nodata': -9999.0, 'width': 8640, 'height': 3432, 'count': 1, 'crs': None, 'transform': Affine(0.041666666666666664, 0.0, -180.0,
       0.0, -0.041666666666666664, 84.984928), 'tiled': False}


RasterioIOError: 'C:/Users/Max81007/Desktop/Python/Resource_Watch/Raster/dis.008/dis_008_vulnerability_to_volcanic_eruptions_edit.tif' does not exist in the file system, and is not recognized as a supported dataset name. 

Use rasterio to reproject and compress

In [5]:
os.getcwd()
os.chdir(local_folder)
os.environ["local_orig"] =local_orig
os.environ["local_edit"] =local_edit

In [6]:
!gdalwarp -overwrite -t_srs epsg:4326 -srcnodata none -co compress=lzw %local_orig% %local_edit%

0...10...20...30...40...50...60...70...80...90...100 - done.


In [7]:
files = [local_orig, local_edit]
for file in files:
    with rio.open(file, 'r') as src:
        profile = src.profile
        print(profile)

{'driver': 'AAIGrid', 'dtype': 'int32', 'nodata': -9999.0, 'width': 8640, 'height': 3432, 'count': 1, 'crs': None, 'transform': Affine(0.041666666666666664, 0.0, -180.0,
       0.0, -0.041666666666666664, 84.984928), 'tiled': False}
{'driver': 'GTiff', 'dtype': 'int32', 'nodata': None, 'width': 8640, 'height': 3432, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.041666666666666664, 0.0, -180.0,
       0.0, -0.041666666666666664, 84.984928), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


Upload orig and edit files to s3

In [8]:
# Original
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))

# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))

C:/Users/Max81007/Desktop/Python/Resource_Watch/Raster/dis.008/dis_008_vulnerability_to_volcanic_eruptions_edit.tif  6779743 / 6779743.0  (100.00%)

In [9]:
os.environ["Zgs_key"] = "gs://resource-watch-public/" + s3_key_orig

In [10]:
!echo %Zs3_key2%
!echo %Zgs_key%

s3://wri-public-data/resourcewatch/raster/dis_008_vulnerability_to_volcanic_eruptions/dis_008_vulnerability_to_volcanic_eruptions_edit.tif
gs://resource-watch-public/resourcewatch/raster/dis_008_vulnerability_to_volcanic_eruptions/dis_008_vulnerability_to_volcanic_eruptions.tif


In [11]:
!gsutil cp %Zs3_key2% %Zgs_key%

Copying s3://wri-public-data/resourcewatch/raster/dis_008_vulnerability_to_volcanic_eruptions/dis_008_vulnerability_to_volcanic_eruptions_edit.tif [Content-Type=binary/octet-stream]...
/ [0 files][    0.0 B/  6.5 MiB]                                                
-
- [0 files][  3.1 MiB/  6.5 MiB]                                                
\
\ [1 files][  6.5 MiB/  6.5 MiB]                                                
|

Operation completed over 1 objects/6.5 MiB.                                      


In [12]:
with rio.open(local_orig) as src:
    data = src.read(indexes=1)
    pyplot.imshow(data)

MemoryError: 

In [ ]:
with rio.open(local_edit) as src:
    data = src.read(indexes=1)
    pyplot.imshow(data)

In [13]:
os.environ["asset_id"] = "users/resourcewatch/dis_008_vulnerability_to_volcanic_eruptions"
!earthengine upload image --asset_id=%asset_id% %Zgs_key%

Started upload task with ID: GCU7G4D4D7HOOCWBXWM3EYFZ


In [14]:
!earthengine task info F7ZP3YOHXBMERJK2KRG4C5M2

F7ZP3YOHXBMERJK2KRG4C5M2:
  State: COMPLETED
  Type: Upload
  Description: Asset ingestion: users/resourcewatch/ene_018_wind_energy_potential
  Created: 2017-10-05 16:37:23.361000
  Started: 2017-10-05 16:37:26.531000
  Updated: 2017-10-05 16:39:03.039000
